In [ ]:
using BSON, CUDA, Dates, DelimitedFiles, Downloads, Flux, Plots #import libraries

include("simulation.jl"); #includes the function 'simulate' used below
include("neural.jl"); #includes 'MINIMZE', 'generate_windows', 'generate_phi'
include("generateVext.jl");

In [ ]:
BSON.@load "model.bson" model #import the neural model
display(model) #summary of the model paramters

In [ ]:
#this function returns the c1 profile for a given density profile and pair potential
function get_c1_neural(model, ϕ) #to define this function, only the length of the discretized pait potential is of importance
    window_bins = length(model.layers[1].weight[1,:])-size(ϕ)[1] # Get the number of input bins from the shape of the first layer
    model = model |> gpu
    function (ρ, ϕ) #ρ and ϕ need to be discretized
        ρ_windows = generate_windows(ρ; window_bins) |> gpu
        ϕ_func = generate_phi(ϕ, ρ) |> gpu
        input = vcat(ρ_windows, ϕ_func)
        model(input) |> cpu |> vec  # Evaluate the model, make sure the result gets back to the CPU, and transpose it to a vector
    end
end

In [ ]:
#some possible pair potentials
Lj(x, ϵ) = 4*ϵ*((σ/x)^12-(σ/x)^6) # LJ

function H(x) #ramp
    if 0 <= x < 1
        return -0.5*x+1.5
    else
        return 0
    end
end

function B(x, ϵ) #box
    if 0 <= x < 1
        return ϵ
    else
        return 0
    end
end

HR(x) = x < 1 ? Inf : 0 #hard rods

 The following cell shows the self-consistent calculation of a density profile dependet on the temperature $T$, the chemical potential $\mu$ (grand canonical ensemble), the external potential $V_{ext}$ and the pair potential $\phi$. We use periodic boundary conditions in a box of size $L$. The DFT-minimization is carried out by the function `MINIMIZE` which performs a Picard-iteration to solve the Euler-Lagrange equation $\rho(x) = \exp (c_1[\rho,\beta \phi] - \beta (V_{ext} - \mu))$ where the neural network (model) is used in place of $c_1$. A simulated density profile is provided by the function `simulate`.   

In [ ]:
L = 5 #length of the box
T = 1.5 #temperature
μ = 1.0 #chemical potential

sin_parameters, lin_parameters, wall_params = Generateparams(L) #random parameters used to construct the external potential
Vext(x) = GenerateV(x, sin_parameters, lin_parameters, wall_params) #generate random external potential

xs1, ρ1 = simulate(L, μ, T, Vext, H; equilibration_time=Dates.Second(1), production_time=Dates.Second(2), sweep_transitions=10)
#increase equilibration_time and production_time for a less noisy profile
xs2, ρ2 = MINIMIZE(L, μ, T, H, Vext, model) #neural prediction
plot(xs1, ρ1, label="simulation", xlabel="x/σ", ylabel="ρ")
plot!(xs2, ρ2, label="neural")